In [1]:
import sys
sys.path.append("../")
import os
import numpy as np
import nibabel as nb
import pandas as pd
import matplotlib.pylab as plt
from data.nifti_dataset import resample_nifti
from data.base_dataset import _roll2center_crop
from scipy.ndimage.measurements import center_of_mass

import DeepStrain.functions_collection as ff


main_path = '/mnt/mount_zc_NAS/Deepstrain'
data_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data'
spreadsheet = pd.read_excel(os.path.join(data_path, 'Patient_list', 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' ))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
patient_list = spreadsheet.iloc[0:50]
result = []
for patient_index in range(0,patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[patient_index]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
    patient_row = spreadsheet.loc[spreadsheet['OurID'] == patient_id_num]
    ED = patient_row['ED'].values[0].astype(int)

    print(patient_id)
    
    folder = os.path.join(main_path, 'results/fine_tune_carson/seg', patient_id)

    seg_files = ff.sort_timeframe(ff.find_all_target_files(['pred_seg*'], folder), 2, 'e', '.')
    
    volume_list = []

    # first load ED to define which slices we should use to compare (mid slices)
    ed_seg = np.round(nb.load(os.path.join(data_path, 'nii_manual_seg', patient_id, 'SAX_ED_seg.nii.gz')).get_fdata()).astype(int)
    # find nonzero slices index
    slice_effective = [i for i in range(ed_seg.shape[2]) if np.sum(ed_seg[:,:,i] == 1)>0]
    compare_slice = [slice_effective[len(slice_effective)//2-1], slice_effective[len(slice_effective)//2], slice_effective[len(slice_effective)//2+1]]


    for tf in range(0,len(seg_files)):
        seg = nb.load(seg_files[tf]).get_fdata()
        volume_list.append(np.sum(seg[:,:,compare_slice] == 1))
    
    volume_list = np.asarray(volume_list)
    print(volume_list)

    # sort the index by its corresponding value from smallest to largest
    sorted_index = np.argsort(volume_list)
    # get the index of the smallest value
    smallest_index = sorted_index[0]

    ES = smallest_index + 1

    result.append([patient_id_num, ED, ES])

    df = pd.DataFrame(result, columns = ['OurID', 'ED', 'ES'])
    df.to_excel(os.path.join(data_path, 'Patient_list/ES_by_AI_seg.xlsx'), index = False)


ID_0692
[2658 2565 2364 2043 1746 1513 1317 1282 1411 1530 1726 2035 2201 2292
 2346 2401 2442 2466 2495 2509 2525 2545 2626 2700 2686]
ID_1207
[3222 2996 2734 2325 2074 1518 1077  953 1048 1071 1123 1237 1487 1806
 2155 2382 2559 2697 2777 2810 2812 2822 2873 3109 3299]
ID_0085
[3202 3102 2891 2635 2382 2094 1848 1660 1555 1526 1625 1819 2007 2356
 2575 2771 2881 2952 3042 3108 3176 3228 3256 3247 3254]
ID_1208
[2996 2889 2605 2357 2135 1880 1647 1446 1248 1166 1214 1406 1577 1653
 1785 1937 2117 2304 2443 2516 2553 2581 2768 3007 3060]
ID_0685
[2884 2876 2836 2661 2380 2119 1874 1719 1652 1705 1789 1910 2064 2277
 2479 2572 2632 2661 2672 2697 2692 2685 2716 2778 2857]
ID_0483
[3416 3349 3152 2861 2550 2244 1906 1668 1486 1560 1679 1699 1786 2005
 2231 2402 2532 2697 2790 2835 2906 3032 3231 3385 3448]
ID_0682
[1477 1411 1361 1257 1109  948  799  683  594  649  726  750  742  783
  902 1044 1140 1185 1209 1204 1195 1196 1241 1346 1491]
ID_0290
[1725 1675 1588 1429 1271 1117  975  853